In [1]:
"""
created on: 24/08/21 23:04
author: @preet
"""

'\ncreated on: 24/08/21 23:04\nauthor: @preet\n'

# Object Detection Using Yolo with OpenCV
I have used the following dataset from kaggle: https://www.kaggle.com/valentynsichkar/yolo-coco-data

In [1]:
# Import the required libraries
import cv2
import numpy as np

In [2]:
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [3]:
# Load class names from 'coco.names'
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
layer_names = net.getLayerNames()
print(layer_names)

['conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'leaky_

In [5]:
# net.getUnconnectedOutLayers()
# net.getUnconnectedOutLayersNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(outputlayers)

['yolo_82', 'yolo_94', 'yolo_106']


In [8]:
# Loading the image
img = cv2.imread("sample_3.jpg")
img = cv2.resize(img, None, fx=0.7, fy=0.7)
height, width, channels = img.shape
colors = np.random.uniform(0, 255, size=(len(classes),3))

In [ ]:
cv2.imshow("Sample Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows

In [10]:
# Detecting Objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop = False)

In [38]:
for b in blob:
    for n,img_blob in enumerate(b):
        cv2.imshow(str(n), img_blob)
        cv2.waitKey(1000)
        cv2.destroyAllWindows

In [11]:
net.setInput(blob)
outs = net.forward(outputlayers)
print(outs)

[array([[0.0376766 , 0.0389263 , 0.27730685, ..., 0.        , 0.        ,
        0.        ],
       [0.04587369, 0.03214624, 0.223802  , ..., 0.        , 0.        ,
        0.        ],
       [0.05648516, 0.03187547, 0.904982  , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95926327, 0.9649099 , 0.5334992 , ..., 0.        , 0.        ,
        0.        ],
       [0.95919263, 0.966611  , 0.3458044 , ..., 0.        , 0.        ,
        0.        ],
       [0.97132564, 0.96748775, 0.8372834 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02276275, 0.01817572, 0.05973727, ..., 0.        , 0.        ,
        0.        ],
       [0.01689335, 0.02119221, 0.29139403, ..., 0.        , 0.        ,
        0.        ],
       [0.02340616, 0.0182234 , 0.08689809, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98127466, 0.9767631 , 0.03796575, ..., 0.        , 0.        ,
        0.        ],
       [0.9811733 

In [12]:
boxes=[]
class_ids=[]
confidences=[]
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            # cv2.circle(img, (center_x, center_y), 10, (0,255,0), 2)
            # Rectangle coordinates
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            boxes.append([x, y, w, h])
            class_ids.append(class_id)
            confidences.append(float(confidence))

In [13]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x, y+30), font, 1, color, 3)